# Parsing WARC File

### 🛠 Installing Necessary Libraries

In [1]:
pip install warcio beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [16]:
from warcio.archiveiterator import ArchiveIterator
from bs4 import BeautifulSoup
import gzip

def extract_html_from_warc(warc_path):
    documents = []
    with open(warc_path, 'rb') as stream:
        for record in ArchiveIterator(stream):
            if record.rec_type == 'response':
                url = record.rec_headers.get_header('WARC-Target-URI')
                raw_html = record.content_stream().read().decode('utf-8', errors='replace')

                soup = BeautifulSoup(raw_html, 'lxml')
                title = soup.title.string if soup.title else ''

                documents.append({
                    'url': url,
                    'title': title,
                    'raw_html': raw_html,
                })

                #if len(documents) >= max_pages:
                #    break
    return documents


In [17]:
data = "./website_data.warc"

In [ ]:
docs = extract_html_from_warc(data)
print(docs.__len__())
# Save the extracted documents to a file